<a href="https://colab.research.google.com/github/20211931/oss_pratice/blob/main/%EC%98%A4%ED%94%88%EC%86%8C%EC%8A%A4_%EC%8B%A4%EC%8A%B57.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

park = Customer('Park', 100)
print(park)


class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


Customer(name='Park', fidelity=100)


In [20]:
class Order:
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '_total'):
            self._total = sum(item.total() for item in self.cart)
        return self._total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)  # self = Order
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

class Promotion(ABC):
    @abstractmethod
    def discount(self, order):
        pass

class FidelityPromo(Promotion):
    def discount(self, order):
        return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

class BulkItemPromo(Promotion):
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * 0.1
        return discount

class LargeOrderPromo(Promotion):
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * 0.07
        return 0

joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]

Order(joe, cart, FidelityPromo())

banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]


long_order = [LineItem(str(item_code), 1, 1.0)
              for item_code in range(10)]


In [19]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [21]:
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [22]:
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [23]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

In [27]:
class Order:

    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '_total'):
            self._total = sum(item.total() for item in self.cart)
        return self._total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

def fidelity_promo(order):
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

def large_order_promo(order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [28]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [29]:
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [30]:
Order(ann, long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

In [32]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
    return max(promo(order) for promo in promos)

Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

In [33]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [34]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

In [35]:
globals().keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', '_i2', 'Order', '_i3', '_i4', 'ABC', 'abstractmethod', 'namedtuple', 'Customer', 'park', 'LineItem', '_i5', '_i6', 'Promotion', 'FidelityPromo', 'BulkItemPromo', 'LargeOrderPromo', '_i7', 'joe', 'ann', 'cart', '_7', '_i8', 'banana_cart', 'long_order', '_8', '_i9', '_i10', '_i11', 'fidelity_promo', 'bulk_item_promo', 'large_order_promo', '_11', '_i12', 'promos', 'best_promo', '_12', '_i13', '_13', '_i14', '_14', '_i15', '_15', '_i16', '_16', '_i17', '_i18', '_i19', '_19', '_i20', '_i21', '_21', '_i22', '_22', '_i23', '_23', '_i24', '_24', '_i25', '_25', '_i26', '_26', '_i27', '_27', '_i28', '_28', '_i29', '_29', '_i30', '_30', '_i31', '_31', '_i32', '_32', '_i33', '_33', '_i34', '_34', '_i35'])

In [36]:
globals()['best_promo']

<function __main__.best_promo(order)>

In [37]:
promos = [globals()[name] for name in globals()
if name.endswith('_promo')
and name != 'best_promo']
promos

[<function __main__.fidelity_promo(order)>,
 <function __main__.bulk_item_promo(order)>,
 <function __main__.large_order_promo(order)>]

In [38]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print("running target()")

target()
target

running inner()


<function __main__.deco.<locals>.inner()>

In [40]:
registry = []

def register(func):
    print(f'running register({func})')
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')

def main():
    import time

    print('running main()')
    print('registry ->', registry)

    f1()
    f2()
    f3()

if __name__ == '__main__':
    main()


running register(<function f1 at 0x79e6c9be8af0>)
running register(<function f2 at 0x79e6c9be8d30>)
running main()
registry -> [<function f1 at 0x79e6c9be8af0>, <function f2 at 0x79e6c9be8d30>]
running f1()
running f2()
running f3()


In [41]:
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity_promo(order):
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item_promo(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

@promotion
def large_order_promo(order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

def best_promo(order):
    return max(promo(order) for promo in promos)


In [42]:
def f1(a):
    print(a)
    print(b)

f1(3)

3


NameError: name 'b' is not defined

In [43]:
b=6
f1(3)

3
6


In [44]:
b=6
def f2(a):
    print(a)
    print(b)
    b=9

f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [47]:
b=6
def f3(a):
    global b
    print(a)
    print(b)
    b=9

f3(3)

print(b)

f3(3)

b=30
f3(3)

3
6
9
3
9
3
30


In [49]:
class Averager():
    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)

avg = Averager()
avg(10)

10.0

In [50]:
avg(11)

10.5

In [51]:
avg(12)

11.0

In [52]:
def make_averager():
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)

    return averager

avg = make_averager()
avg(10)

10.0

In [53]:
avg(11)

10.5

In [54]:
avg(12)

11.0

In [55]:
avg.__code__.co_varnames

('new_value', 'total')

In [56]:
avg.__code__.co_freevars

('series',)

In [57]:
avg.__closure__

(<cell at 0x79e6c9e72050: list object at 0x79e6c9bfae80>,)

In [58]:
avg.__closure__[0].cell_contents

[10, 11, 12]

In [60]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        count += 1
        total += new_value
        return total / count

    return averager

avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [61]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count

    return averager

avg = make_averager()
avg(10)

10.0

In [64]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result}')
        return result
    return clocked

In [63]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
    print('*'*40, 'Calling snooze(.123)')
    snooze(.123)
    print('*'*40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12318596s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000052s] factorial(1) -> 1
[0.00005810s] factorial(2) -> 2
[0.00009259s] factorial(3) -> 6
[0.00012359s] factorial(4) -> 24
[0.00015294s] factorial(5) -> 120
[0.00018617s] factorial(6) -> 720
6! = 720


In [65]:
factorial.__name__

'clocked'

In [66]:
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result}')
        return result
    return clocked

In [67]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
    print('*'*40, 'Calling snooze(.123)')
    snooze(.123)
    print('*'*40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12322807s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000072s] factorial(1) -> 1
[0.00005007s] factorial(2) -> 2
[0.00008368s] factorial(3) -> 6
[0.00011253s] factorial(4) -> 24
[0.00014234s] factorial(5) -> 120
[0.00017214s] factorial(6) -> 720
6! = 720


In [68]:
factorial.__name__

'factorial'